In [ ]:
import os
import nibabel as nib
import numpy as np
from skimage.transform import resize

# Define the input directories for 3D medical images and binary masks
images_dir = "/kaggle/input/fortrain1/nifti_images"
masks_dir = "/kaggle/working/mask"

# Define the output directory for cropped images
output_dir = "/kaggle/working/cro"
os.makedirs(output_dir, exist_ok=True)

# Function to perform masked cropping
def masked_crop(image_path, mask_path, output_path, target_shape=None):
    # Load the 3D medical image and binary mask
    image = nib.load(image_path).get_fdata()
    mask = nib.load(mask_path).get_fdata()

    # Check if the shapes of the image and mask match
    if image.shape != mask.shape:
        raise ValueError("Image and mask shapes do not match!")

    # Create a binary mask where non-black (non-empty) regions are set to 1
    binary_mask = (mask > 0).astype(image.dtype)

    # Apply element-wise multiplication to mask the image
    masked_image = image * binary_mask

    # Optionally, crop or resize the masked image to a fixed size or shape
    if target_shape:
        masked_image = resize(masked_image, target_shape, preserve_range=True, anti_aliasing=False)

    # Create a new NIfTI image from the cropped/resized array
    cropped_image_nifti = nib.Nifti1Image(masked_image, np.eye(4))

    # Save the cropped/resized image to the output directory
    nib.save(cropped_image_nifti, output_path)

# Iterate through each image file
for image_file in os.listdir(images_dir):
    if image_file.endswith(".nii"):
        image_path = os.path.join(images_dir, image_file)
        
        # Derive the corresponding mask filename
        mask_file = image_file.replace(".nii", ".nii")
        mask_path = os.path.join(masks_dir, mask_file)
        output_path = os.path.join(output_dir, image_file)  # Change this as needed

        # Perform masked cropping with optional resizing (e.g., target_shape=(128, 128, 128))
        masked_crop(image_path, mask_path, output_path, target_shape=None)
        print(f"Cropped and saved {output_path}")

print("Masked cropping complete.")


In [ ]:
import os
import shutil
import nibabel as nib
import numpy as np

# Define the source and destination directories
source_dir = "/kaggle/input/fortrain-mask1"
destination_dir = "/kaggle/working/mask"

# Create the destination directory if it doesn't exist
os.makedirs(destination_dir, exist_ok=True)

# Iterate through files in the source directory
for filename in os.listdir(source_dir):
    # Check if the file ends with "_mask.nii" to identify mask files
    if filename.endswith("_mask.nii"):
        # Remove "_mask" from the filename
        new_filename = filename.replace("_mask.nii", ".nii")

        # Create the full paths for source and destination files
        source_path = os.path.join(source_dir, filename)
        destination_path = os.path.join(destination_dir, new_filename)

        # Load the mask
        mask_img = nib.load(source_path)
        mask_data = mask_img.get_fdata()

        # Threshold the mask to create a binary mask (0 for background, 1 for foreground)
        binary_mask_data = (mask_data > 0).astype(np.uint8)

        # Create a new NIfTI image from the binary mask
        binary_mask_img = nib.Nifti1Image(binary_mask_data, mask_img.affine)

        # Save the binary mask to the destination directory with the new name
        nib.save(binary_mask_img, destination_path)

        print(f"Copied, thresholded, and renamed: {source_path} -> {destination_path}")

print("Mask files copied, thresholded, and renamed.")


In [ ]:
import nibabel as nib
import matplotlib.pyplot as plt

# Define the path to the NII image
nii_path = '/kaggle/working/jjj.nii.gz'

# Load the NII image
nii_img = nib.load(nii_path)
nii_data = nii_img.get_fdata()

# Define the range of slices you want to view (from 0 to 60)
start_slice = 0
end_slice = 360

# Calculate the number of rows needed
num_slices = end_slice - start_slice
num_rows = (num_slices + 4) // 5  # 5 images per row

# Create subplots for the rows
fig, axs = plt.subplots(num_rows, 5, figsize=(15, 5 * num_rows))

# Plot the slices
for row in range(num_rows):
    for col in range(5):
        slice_num = start_slice + row * 5 + col
        if slice_num < end_slice:
            slice_data = nii_data[:, :, slice_num]  # Extract the 2D slice data
            axs[row, col].imshow(slice_data, cmap='gray')  # Display the slice
            axs[row, col].set_title(f'Slice {slice_num}')
            axs[row, col].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import os
import nibabel as nib
import numpy as np

# Define the directory containing the cropped medical images
directory_path = '/kaggle/working/cro'

# List all NIfTI files in the directory
nifti_files = [file for file in os.listdir(directory_path) if file.endswith('.nii')]

# Function to calculate the percentage of cropping for an image
def calculate_cropping_percentage(image_data):
    total_voxels = np.prod(image_data.shape)
    non_zero_voxels = np.count_nonzero(image_data)
    cropping_percentage = (1 - (non_zero_voxels / total_voxels)) * 100
    return cropping_percentage

# Iterate through each NIfTI file, delete completely dark slices, and calculate cropping percentage
for file_name in nifti_files:
    file_path = os.path.join(directory_path, file_name)
    nifti_img = nib.load(file_path)
    nifti_data = nifti_img.get_fdata()
    
    # Calculate cropping percentage
    cropping_percentage = calculate_cropping_percentage(nifti_data)
    
    # Delete completely dark slices
    cropped_data = np.delete(nifti_data, np.where(np.all(nifti_data == 0, axis=(0, 1))), axis=2)
    
    # Create a new NIfTI image from the cropped data
    cropped_img = nib.Nifti1Image(cropped_data, nifti_img.affine)
    
    # Save the cropped image back to the same file
    nib.save(cropped_img, file_path)
    
    print(f"Processed {file_name}: Cropping Percentage = {cropping_percentage:.2f}%")


In [ ]:
1+1

In [ ]:
import nibabel as nib
import numpy as np
import matplotlib.pyplot as plt

# Define the paths to the NII images
image_path   = '/kaggle/working/nifti_images/mirsanmir_rotated.nii'
mask_path = '/kaggle/input/rsna-2023-abdominal-trauma-detection/segmentations/10000.nii'

# Load the NII images
image_nii = nib.load(image_path)
mask_nii = nib.load(mask_path)

# Get the image data and mask data
image_data = image_nii.get_fdata()
mask_data = mask_nii.get_fdata()

# Rotate the data 90 degrees counterclockwise
image_data = np.rot90(image_data, k=1, axes=(0, 1))
mask_data = np.rot90(mask_data, k=1, axes=(0, 1))

# Define the range of slices you want to view (from 0 to 60)
start_slice = 0
end_slice = 450

# Calculate the number of rows needed
num_slices = end_slice - start_slice
num_rows = (num_slices + 4) // 5  # 5 images per row

# Create subplots for the rows
fig, axs = plt.subplots(num_rows, 5, figsize=(15, 5 * num_rows))

# Plot the slices
for row in range(num_rows):
    for col in range(5):
        slice_num = start_slice + row * 5 + col
        if slice_num < end_slice:
            image_slice = image_data[:, :, slice_num]  # Extract the 2D image slice data
            mask_slice = mask_data[:, :, slice_num]  # Extract the 2D mask slice data

            # Overlay the mask on the image using a colormap
            overlayed_slice = np.zeros_like(image_slice)
            overlayed_slice[mask_slice > 0] = 0.5 * image_slice[mask_slice > 0]  # Adjust overlay intensity as needed

            axs[row, col].imshow(image_slice, cmap='gray')  # Display the image
            axs[row, col].imshow(overlayed_slice, cmap='jet', alpha=0.1)  # Overlay the mask
            axs[row, col].set_title(f'Slice {slice_num}')
            axs[row, col].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
pip install dicom2nifti


In [ ]:
import os
import pydicom
import nibabel as nib
import numpy as np
from skimage.transform import resize

# Define the data directory
data_directory = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/54722/10000"

# Create a directory for saving the NIfTI image
output_directory = "/kaggle/working/nifti_image"
os.makedirs(output_directory, exist_ok=True)

# Function to load and resize DICOM images from a directory
def load_and_resize_dicom_images(dicom_dir, target_shape):
    dicom_files = [os.path.join(dicom_dir, filename) for filename in os.listdir(dicom_dir)]
    dicom_files.sort(key=lambda x: pydicom.dcmread(x).InstanceNumber)
    dicom_images = [pydicom.dcmread(file).pixel_array for file in dicom_files]
    resized_images = [resize(image, target_shape, preserve_range=True, anti_aliasing=False) for image in dicom_images]
    return resized_images

# Load and resize DICOM images to (128, 128)
resized_dicom_images = load_and_resize_dicom_images(data_directory, target_shape=(512, 512))

# Create a 3D NIfTI image with the shape of the DICOM images
nifti_image = np.stack(resized_dicom_images, axis=-1)
nifti_image = nib.Nifti1Image(nifti_image, np.eye(4))

# Save the NIfTI image in the output directory
nifti_save_path = os.path.join(output_directory, "output.nii")
nib.save(nifti_image, nifti_save_path)

print(f"Created NIfTI image and saved it to {nifti_save_path}")


In [ ]:
import nibabel as nib
import numpy as np

# Load the original NIfTI image
nii_path = '/kaggle/working/nifti_image/output.nii'
nii_img = nib.load(nii_path)
nii_data = nii_img.get_fdata()

# Rotate the data 90 degrees anticlockwise
rotated_data = np.rot90(nii_data, k=1, axes=(1, 0))

# Create a new NIfTI image with the rotated data
rotated_nii_img = nib.Nifti1Image(rotated_data, nii_img.affine)

# Save the rotated NIfTI image
output_path = '/kaggle/working/nifti_image/output_rotated_anticlockwise.nii'
nib.save(rotated_nii_img, output_path)

print(f"Rotated NIfTI image anticlockwise and saved it to {output_path}")



In [ ]:
import os
import pydicom
import nibabel as nib
import numpy as np
from skimage.transform import resize

# Define the path to the DICOM directory
dicom_dir = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/54722/10000"

# Create a directory for saving the NIfTI images
output_directory = "/kaggle/working/nifti_images"
os.makedirs(output_directory, exist_ok=True)

# Function to load and resize DICOM images from a directory
def load_and_resize_dicom_images(dicom_dir, target_shape):
    dicom_files = [os.path.join(dicom_dir, filename) for filename in os.listdir(dicom_dir)]
    dicom_files.sort()  # Sort DICOM files in ascending order
    dicom_images = [pydicom.dcmread(file).pixel_array for file in dicom_files]
    dicom_images.reverse()  # Reverse the order of DICOM images
    resized_images = [resize(image, target_shape, preserve_range=True, anti_aliasing=False) for image in dicom_images]
    return resized_images

# Iterate through each DICOM file in the directory
dicom_files = [os.path.join(dicom_dir, filename) for filename in os.listdir(dicom_dir)]
dicom_files.sort()  # Sort DICOM files in ascending order

# Load and resize DICOM images to (128, 128)
resized_dicom_images = load_and_resize_dicom_images(dicom_dir, target_shape=(512, 512))

# Create a 3D NIfTI image with the shape of the DICOM images
nifti_image = np.stack(resized_dicom_images, axis=-1)
nifti_image = nib.Nifti1Image(nifti_image, np.eye(4))

# Set NIfTI header spacing based on a reference DICOM image
reference_dicom = pydicom.dcmread(dicom_files[0])  # Use the first DICOM file as a reference
nifti_image.header.set_zooms((reference_dicom.PixelSpacing[0], reference_dicom.PixelSpacing[1], reference_dicom.SliceThickness))

# Save the NIfTI image
output_path = os.path.join(output_directory, "mirsanmir.nii")
nib.save(nifti_image, output_path)

print(f"Created NIfTI image with flipped slices and saved it to {output_path}")


In [ ]:
import os
import pydicom
import nibabel as nib
import numpy as np
from skimage.transform import resize, rotate

# Define the path to the DICOM directory
dicom_dir = "/kaggle/input/rsna-2023-abdominal-trauma-detection/train_images/54722/10000"

# Create a directory for saving the NIfTI images
output_directory = "/kaggle/working/nifti_images"
os.makedirs(output_directory, exist_ok=True)

# Function to load and resize DICOM images from a directory
def load_and_resize_dicom_images(dicom_dir, target_shape):
    dicom_files = [os.path.join(dicom_dir, filename) for filename in os.listdir(dicom_dir)]
    dicom_files.sort()  # Sort DICOM files in ascending order
    dicom_images = [pydicom.dcmread(file).pixel_array for file in dicom_files]
    dicom_images.reverse()  # Reverse the order of DICOM images
    resized_images = [resize(image, target_shape, preserve_range=True, anti_aliasing=False) for image in dicom_images]
    return resized_images

# Iterate through each DICOM file in the directory
dicom_files = [os.path.join(dicom_dir, filename) for filename in os.listdir(dicom_dir)]
dicom_files.sort()  # Sort DICOM files in ascending order

# Load and resize DICOM images to (128, 128)
resized_dicom_images = load_and_resize_dicom_images(dicom_dir, target_shape=(512, 512))

# Create a 3D NIfTI image with the shape of the DICOM images
nifti_image = np.stack(resized_dicom_images, axis=-1)
nifti_image = nib.Nifti1Image(nifti_image, np.eye(4))

# Set NIfTI header spacing based on a reference DICOM image
reference_dicom = pydicom.dcmread(dicom_files[0])  # Use the first DICOM file as a reference
nifti_image.header.set_zooms((reference_dicom.PixelSpacing[0], reference_dicom.PixelSpacing[1], reference_dicom.SliceThickness))

# Rotate the NIfTI image 90 degrees clockwise
nifti_image_data = np.rot90(nifti_image.get_fdata(), k=3, axes=(0, 1))  # Rotate 90 degrees clockwise
nifti_image = nib.Nifti1Image(nifti_image_data, nifti_image.affine)

# Save the rotated NIfTI image
output_path = os.path.join(output_directory, "mirsanmir_rotated.nii")
nib.save(nifti_image, output_path)

print(f"Created NIfTI image with flipped slices and rotated clockwise 90 degrees, saved to {output_path}")
